# **Multi-Dimensional Candidate Matching System for Any Job Posting**

## **1. Installations**

In [1]:

print("Installing required libraries...")
!pip install openai -q
!pip install pydantic -q
!pip install sentence-transformers -q
!pip install rank-bm25 -q
!pip install chromadb -q
!pip install tenacity -q
!pip install pdfplumber -q
!pip install python-docx -q
!pip install streamlit -q
!pip install pyngrok -q
!pip install numpy -q
print("Installation complete.")

Installing required libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **2. Imports & API Key**

In [16]:

import os
import json
import pdfplumber
import docx
import re
import numpy as np
from getpass import getpass

OPENROUTER_API_KEY = getpass("Enter your OpenRouter API Key: ")
os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY

print("API Key set.")

Enter your OpenRouter API Key: ··········
API Key set.


# **config.py**

In [3]:
%%writefile config.py

from pydantic import BaseModel, Field
from typing import List, Dict

LLM_PARSING_MODEL = "mistralai/mistral-7b-instruct:free"
LLM_EXPLAIN_MODEL = "mistralai/mistral-7b-instruct:free"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

TOP_K_RETRIEVAL = 10

SCORING_WEIGHTS = {
    "must_have_skills": 0.35,
    "important_skills": 0.25,
    "nice_to_have_skills": 0.10,
    "experience_relevance": 0.15,
    "recency": 0.10,
    "domain_match": 0.05
}


class SkillCluster(BaseModel):
    must_have: List[str]
    important: List[str]
    nice_to_have: List[str]
    implicit_skills: List[str]

class ParsedJob(BaseModel):
    job_title: str
    required_years_experience: int
    skills: SkillCluster
    domain_keywords: List[str] = Field(default_factory=list, description="Industry keywords (e.g., 'FinTech', 'Healthcare')")
    responsibilities_summary: str

class ExperienceEntry(BaseModel):
    title: str
    company: str
    start_date: str
    end_date: str
    description: str

class ParsedResume(BaseModel):
    name: str
    total_years_experience: int
    skills: List[str]
    education: List[str] = Field(default_factory=list, description="Degrees or certifications (e.g., 'B.S. in Computer Science')")
    domain_keywords: List[str] = Field(default_factory=list, description="Industry keywords (e.g., 'FinTech', 'E-commerce')")
    experience: List[ExperienceEntry]
    full_text_summary: str

class Explanation(BaseModel):
    overall_fit_score: int = Field(..., description="The final score from 0-100")
    confidence: str = Field(..., description="High, Medium, or Low")
    strengths: str = Field(..., description="A string (use bullet points) of what the candidate matches well.")
    gaps: str = Field(..., description="A string (use bullet points) of what the candidate seems to be missing.")
    notes: str = Field(..., description="A brief, overall recommendation.")


# -System Prompts -
PROMPT_PARSE_JOB = """
You are an expert JSON-generating HR analyst. You must analyze the job posting
and return *only* a valid JSON object. The JSON object must strictly
adhere to the following Pydantic JSON Schema:

```json
{schema}
```

Job Posting Text:
---
{job_text}
---

Respond *only* with the JSON. Do not add any other text.
"""

PROMPT_PARSE_RESUME = """
You are an expert JSON-generating resume parser. You must analyze the resume
and return *only* a valid JSON object. The JSON object must strictly
adhere to the following Pydantic JSON Schema:

```json
{schema}
```

Resume Text:
---
{resume_text}
---

Respond *only* with the JSON. Do not add any other text.
"""

PROMPT_EXPLAIN_MATCH = """
You are a helpful recruiting assistant. A job posting and a candidate's profile
have been scored across 6 dimensions. Your job is to generate a final report
as a JSON object.

**Data:**
- Job Summary: {job_summary}
- Resume Summary: {resume_summary}
- Score (Must-Have): {must_have_score}/100
- Score (Important): {important_score}/100
- Score (Experience): {experience_score}/100
- Score (Recency): {recency_score}/100
- Score (Domain): {domain_score}/100
- Final Score: {final_score}/100

Your task is to generate a final report for the hiring manager.
The report must be a JSON object with 5 keys:
'overall_fit_score' (int), 'confidence' (string), 'strengths' (string),
'gaps' (string), and 'notes' (string).
"""
print("File 'config.py' created.")

Writing config.py


# **llm_interface.py**

In [4]:
%%writefile llm_interface.py

import os
import json
from openai import OpenAI
from pydantic import BaseModel
from typing import Type
import config

from tenacity import retry, stop_after_attempt, wait_random_exponential

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def robust_llm_call(prompt: str, response_model: Type[BaseModel], model: str):
    """
    A robust function to call an LLM and parse the output into a Pydantic model
    using 'response_format'.
    """
    try:
        print(f"LLM Call: Pydantic parsing with {model}...")

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )

        content = response.choices[0].message.content
        arguments = json.loads(content)

        return response_model.model_validate(arguments)

    except Exception as e:
        print(f"Error in LLM call or JSON parsing: {e}")
        print(f"Model: {model}, Prompt: {prompt[:100]}...")
        raise e

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def generative_llm_call(prompt: str, model: str):
    """
    A robust function for a simple generative LLM call expecting JSON.
    """
    try:
        print(f"LLM Call: Generative explanation with {model}...")

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Error in generative LLM call: {e}")
        raise e

def parse_job_posting(job_text: str) -> config.ParsedJob:
    """Uses LLM to structure a job posting."""
    schema = config.ParsedJob.model_json_schema()
    prompt = config.PROMPT_PARSE_JOB.format(
        schema=json.dumps(schema, indent=2),
        job_text=job_text
    )
    return robust_llm_call(prompt, config.ParsedJob, config.LLM_PARSING_MODEL)

def parse_resume(resume_text: str) -> config.ParsedResume:
    """Uses LLM to structure a resume."""
    schema = config.ParsedResume.model_json_schema()
    prompt = config.PROMPT_PARSE_RESUME.format(
        schema=json.dumps(schema, indent=2),
        resume_text=resume_text
    )
    return robust_llm_call(prompt, config.ParsedResume, config.LLM_PARSING_MODEL)

def generate_explanation(report_data: dict) -> dict:
    """Uses LLM to generate the final human-readable report."""
    prompt = config.PROMPT_EXPLAIN_MATCH.format(**report_data)

    try:
        explanation_json = generative_llm_call(prompt, config.LLM_EXPLAIN_MODEL)
        report_data.update(explanation_json)
        return report_data
    except Exception as e:
        print(f"Error in explanation LLM call (final attempt failed): {e}")
        report_data.update({
            "strengths": "Error generating AI analysis.",
            "gaps": "Error generating AI analysis.",
            "notes": "System error."
        })
        return report_data

print("File 'llm_interface.py' (Final Version with Import Fix) created.")

Writing llm_interface.py


# **scoring.py**

In [5]:
%%writefile scoring.py

import config
import re
from datetime import datetime

class ScoringEngine:
    def __init__(self):
        self.weights = config.SCORING_WEIGHTS
        print("ScoringEngine initialized.")

    def _score_skills(self, required: list, candidate_keywords: list) -> float:
        """
        --- CORRECTED LOGIC ---
        Scores skill match from 0 to 100.
        Checks if any candidate keyword (e.g., "Python", "B.S. in CS") exists
        inside the job requirement phrase (e.g., "5+ years of Python experience").
        """
        if not required:
            return 100.0

        matched_count = 0
        candidate_keywords_lower = [s.lower() for s in candidate_keywords]

        for req_phrase in required:
            req_phrase_lower = req_phrase.lower()
            for skill_keyword in candidate_keywords_lower:
                if skill_keyword in req_phrase_lower:
                    matched_count += 1
                    break

        return (matched_count / len(required)) * 100

    def _score_experience(self, required_years: int, candidate_years: int) -> float:
        """Scores experience match from 0 to 100 (capped at 100)."""
        if required_years == 0:
            return 100.0

        if candidate_years >= required_years:
            return 100.0
        else:
            return (candidate_years / required_years) * 100

    def _score_recency(self, experience_entries: list) -> float:
        """Scores recency. 100 if a job ended in the last 2 years or is 'Present'."""
        if not experience_entries:
            return 0.0

        today = datetime.today()
        for entry_data in experience_entries:
            end_date = entry_data.get('end_date', '').lower()
            if end_date == 'present':
                return 100.0

            try:
                end_year_match = re.search(r'(\d{4})', end_date)
                if end_year_match:
                    end_year = int(end_year_match.group(1))
                    if (today.year - end_year) <= 2:
                        return 100.0
            except:
                continue

        return 30.0

    def _score_domain(self, job_keywords: list, resume_keywords: list) -> float:
        """
        --- NEW, ROBUST LOGIC ---
        Scores domain match. Checks for any overlap between keyword lists.
        """
        if not job_keywords:
            return 50.0

        job_set = set(k.lower() for k in job_keywords)
        resume_set = set(k.lower() for k in resume_keywords)

        if job_set.intersection(resume_set):
            return 100.0
        else:
            return 0.0

    def score_candidate(self, job: config.ParsedJob, resume: config.ParsedResume) -> dict:
        """Runs the full 6-dimension scoring for a single candidate."""
        print(f"Re-ranking candidate: {resume.name}")

        job_skills = job.skills.model_dump()
        resume_experience = [exp.model_dump() for exp in resume.experience]

        candidate_keywords = resume.skills + resume.education

        scores = {
            "must_have_score": self._score_skills(job_skills['must_have'], candidate_keywords),
            "important_score": self._score_skills(job_skills['important'], candidate_keywords),
            "nice_to_have_score": self._score_skills(job_skills['nice_to_have'], candidate_keywords),
            "experience_score": self._score_experience(job.required_years_experience, resume.total_years_experience),
            "recency_score": self._score_recency(resume_experience),
            "domain_score": self._score_domain(job.domain_keywords, resume.domain_keywords)
        }

        final_score = (
            scores["must_have_score"] * self.weights["must_have_skills"] +
            scores["important_score"] * self.weights["important_skills"] +
            scores["nice_to_have_score"] * self.weights["nice_to_have_skills"] +
            scores["experience_score"] * self.weights["experience_relevance"] +
            scores["recency_score"] * self.weights["recency"] +
            scores["domain_score"] * self.weights["domain_match"]
        )

        report_data = {
            "name": resume.name,
            "final_score": round(final_score, 2),
            "job_summary": job.responsibilities_summary,
            "resume_summary": resume.full_text_summary,
            **scores
        }

        return report_data

print("File 'scoring.py' (Final Version with Corrected Logic) created.")

Writing scoring.py


# **utils.py**

In [6]:
%%writefile utils.py

import pdfplumber
import docx
import os
import re
from typing import List, Union

def extract_text(file_path: str) -> Union[str, None]:
    """Extracts text from PDF, DOCX, or TXT."""
    print(f"Extracting text from: {file_path}")
    _, extension = os.path.splitext(file_path)

    try:
        if extension == '.pdf':
            with pdfplumber.open(file_path) as pdf:
                return "\n".join(page.extract_text() for page in pdf.pages)
        elif extension == '.docx':
            doc = docx.Document(file_path)
            return "\n".join(para.text for para in doc.paragraphs)
        elif extension == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            print(f"Warning: Unsupported file type: {file_path}")
            return None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

def simple_tokenizer(text: str) -> List[str]:
    """A simple tokenizer for BM25."""
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text.split()

print("File 'utils.py' created.")

Writing utils.py


# **retrieval.py**

In [7]:
%%writefile retrieval.py

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import numpy as np
import config
from utils import simple_tokenizer
import chromadb
from chromadb.utils import embedding_functions

class HybridRetriever:
    def __init__(self):
        self.bm25_index = None
        self.corpus_ids = []

        self.sbert_model = SentenceTransformer(config.EMBEDDING_MODEL)

        self.chroma_client = chromadb.Client()

        self.embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=config.EMBEDDING_MODEL
        )

        self.collection = self.chroma_client.get_or_create_collection(
            name="resume_collection",
            embedding_function=self.embedding_func
        )
        print("HybridRetriever initialized with ChromaDB.")

    def index(self, corpus: list[str], corpus_ids: list[str]):
        """Creates the BM25 and ChromaDB indexes."""
        if not corpus:
            print("Warning: No documents to index.")
            return

        print(f"Indexing {len(corpus)} documents...")
        self.corpus_ids = corpus_ids

        tokenized_corpus = [simple_tokenizer(doc) for doc in corpus]
        self.bm25_index = BM25Okapi(tokenized_corpus)


        existing_ids = self.collection.get()['ids']
        if existing_ids:
            print(f"Clearing {len(existing_ids)} old items from ChromaDB.")
            self.collection.delete(ids=existing_ids)

        self.collection.add(
            documents=corpus,
            ids=corpus_ids
        )
        print("Indexing complete.")

    def search(self, query: str, top_k: int) -> list[str]:
        """
        Performs hybrid search.
        1. Gets top_k from Sparse (BM25).
        2. Gets top_k from Dense (Chroma).
        3. Returns the UNION of the two lists.
        """
        if self.bm25_index is None:
            raise Exception("Must call .index() before .search()")

        print(f"Running hybrid search for query: {query[:50]}...")

        tokenized_query = simple_tokenizer(query)
        bm25_scores = self.bm25_index.get_scores(tokenized_query)

        bm25_top_indices = np.argsort(bm25_scores)[::-1][:top_k]
        sparse_ids = [self.corpus_ids[i] for i in bm25_top_indices]
        print(f"BM25 found IDs: {sparse_ids}")

        dense_results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )
        dense_ids = dense_results['ids'][0]
        print(f"ChromaDB found IDs: {dense_ids}")

        fused_ids = list(set(sparse_ids) | set(dense_ids))

        print(f"Retrieval found {len(fused_ids)} unique candidates for re-ranking.")
        return fused_ids

print("File 'retrieval.py' (Upgraded with ChromaDB Fix) created.")

Writing retrieval.py


# **matching_system.py**

In [8]:
%%writefile matching_system.py

# The main orchestrator class that runs the full pipeline.

import utils
import llm_interface
from retrieval import HybridRetriever
from scoring import ScoringEngine
import config
from typing import List
import traceback

class CandidateMatchingSystem:
    def __init__(self):
        self.job = None
        self.candidates_db = {}
        self.retriever = HybridRetriever()
        self.scorer = ScoringEngine()
        print("CandidateMatchingSystem initialized.")

    def process_job_posting(self, job_file: str):
        """Loads and parses the job posting."""
        job_text = utils.extract_text(job_file)
        if not job_text:
            raise Exception("Job posting file is empty or unreadable.")

        self.job = llm_interface.parse_job_posting(job_text)
        if not self.job:
            raise Exception("LLM failed to parse job posting.")

        print(f"Successfully parsed job: {self.job.job_title}")

    def process_resumes(self, resume_files: List[str]):
        """Loads and parses all candidate resumes."""
        self.candidates_db = {}

        print(f"Starting processing for {len(resume_files)} resumes...")

        for f in resume_files:
            try:
                print(f"Processing file: {f}")
                resume_text = utils.extract_text(f)
                if not resume_text:
                    print(f"Skipping empty or unreadable file: {f}")
                    continue

                parsed_resume = llm_interface.parse_resume(resume_text)
                if parsed_resume:
                    file_id = f.split('/')[-1]
                    self.candidates_db[file_id] = parsed_resume
                    print(f"Successfully parsed: {file_id}")
                else:
                    print(f"LLM failed to parse resume: {f}")
            except Exception as e:
                print(f"Error processing resume {f}: {str(e)}")
                traceback.print_exc()
                continue

        print(f"Successfully parsed {len(self.candidates_db)} out of {len(resume_files)} resumes.")

    def run_matching_pipeline(self) -> List[dict]:
        """
        Runs the full retrieve -> re-rank -> explain pipeline.
        """
        if not self.job or not self.candidates_db:
            raise Exception("Job and resumes must be processed first.")

        corpus = [res.full_text_summary for res in self.candidates_db.values()]
        corpus_ids = list(self.candidates_db.keys())

        if not corpus:
             return []

        self.retriever.index(corpus, corpus_ids)


        k_to_retrieve = min(len(corpus_ids), config.TOP_K_RETRIEVAL)
        query = self.job.responsibilities_summary

        candidate_ids_to_rank = self.retriever.search(query, top_k=k_to_retrieve)

        print(f"Re-ranking {len(candidate_ids_to_rank)} candidates...")

        reports = []
        for candidate_id in candidate_ids_to_rank:
            candidate = self.candidates_db.get(candidate_id)
            if not candidate:
                print(f"Warning: Could not find candidate for ID {candidate_id}")
                continue

            try:
                report = self.scorer.score_candidate(self.job, candidate)
                report["filename"] = candidate_id
                reports.append(report)
            except Exception as e:
                print(f"Error scoring candidate {candidate_id}: {e}")
                continue

        explained_reports = []
        for report in reports:
            try:
                explained_report = llm_interface.generate_explanation(report)
                explained_reports.append(explained_report)
            except Exception as e:
                 print(f"Error generating explanation for {report.get('filename')}: {e}")
                 explained_reports.append(report)

        sorted_reports = sorted(explained_reports, key=lambda r: r['final_score'], reverse=True)

        return sorted_reports

print("File 'matching_system.py' (Final Robust Version) created.")

Writing matching_system.py


# **app.py**

In [9]:
%%writefile app.py

import streamlit as st
import os
import importlib
import time
import sys

sys.path.append(os.getcwd())

# Set page config to wide mode and add a title/icon
st.set_page_config(
    page_title="TalentScout AI | Smart Hiring Assistant",
    page_icon="🚀",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Custom CSS for Professional Look ---
st.markdown("""
    <style>
    .main {
        background-color: #f8f9fa;
    }
    .stButton>button {
        width: 100%;
        border-radius: 5px;
        height: 3em;
        background-color: #4CAF50;
        color: white;
        font-weight: bold;
    }
    .stButton>button:hover {
        background-color: #45a049;
        border-color: #45a049;
        color: white;
    }
    .report-card {
        background-color: white;
        padding: 20px;
        border-radius: 10px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        margin-bottom: 20px;
        border-left: 5px solid #4CAF50;
    }
    .metric-box {
        background-color: #f1f3f4;
        padding: 10px;
        border-radius: 5px;
        text-align: center;
    }
    h1, h2, h3 {
        color: #2c3e50;
    }
    </style>
    """, unsafe_allow_html=True)


def save_uploaded_file(uploaded_file, save_dir="data"):
    """Saves an uploaded file to a directory."""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


    filename, ext = os.path.splitext(uploaded_file.name)
    file_path = os.path.join(save_dir, f"{filename}_{int(time.time())}{ext}")

    with open(file_path, "wb") as f:
        f.write(uploaded_file.getbuffer())
    return file_path

@st.cache_resource
def get_system():
    """Caches the main system class."""
    import config, utils, llm_interface, retrieval, scoring, matching_system
    importlib.reload(config)
    importlib.reload(utils)
    importlib.reload(llm_interface)
    importlib.reload(retrieval)
    importlib.reload(scoring)
    importlib.reload(matching_system)

    from matching_system import CandidateMatchingSystem
    return CandidateMatchingSystem()

# --- 2. Sidebar (Configuration) ---
with st.sidebar:
    st.image("https://cdn-icons-png.flaticon.com/512/4712/4712009.png", width=60)
    st.title("TalentScout AI")
    st.markdown("---")

    st.subheader("1. Configuration")
    api_key = st.text_input("OpenRouter API Key", type="password", help="Enter your API key to enable the AI engine.")
    if api_key:
        os.environ['OPENROUTER_API_KEY'] = api_key
        st.success("✅ API Key Connected")

    st.subheader("2. Job Details")
    job_file = st.file_uploader("Upload Job Description", type=["pdf", "docx", "txt"], help="Upload the JD to match candidates against.")

    st.subheader("3. Candidate Pool")
    resume_files = st.file_uploader("Upload Resumes", type=["pdf", "docx", "txt"], accept_multiple_files=True, help="Select multiple resumes to analyze.")

    st.markdown("---")
    run_button = st.button("🚀 Start Matching Analysis")

    st.markdown("---")
    st.caption("Powered by Mistral 7B & Vector Search")


st.title("🚀 AI Candidate Matching Dashboard")
st.markdown("### Intelligent ranking based on skills, experience, and semantic relevance.")

if run_button:
    if not api_key:
        st.warning("⚠️ Please enter your OpenRouter API Key in the sidebar to proceed.")
    elif not job_file:
        st.warning("⚠️ Please upload a Job Description document.")
    elif not resume_files:
        st.warning("⚠️ Please upload at least one candidate resume.")
    else:
        status_container = st.container()

        try:
            with status_container:
                st.info("🔄 Initializing AI Engine...")
                st.cache_resource.clear()
                system = get_system()

            with status_container:
                with st.spinner("📄 Analyzing Job Description..."):
                    job_path = save_uploaded_file(job_file, "data/job")
                    system.process_job_posting(job_path)
                st.success(f"✅ Job Processed: **{system.job.job_title}**")

                with st.spinner(f"👥 Analyzing {len(resume_files)} Resumes... (This may take a moment)"):
                    resume_paths = [save_uploaded_file(f, "data/resumes") for f in resume_files]
                    system.process_resumes(resume_paths)
                st.success(f"✅ {len(system.candidates_db)} Candidates Analyzed")

            with status_container:
                with st.spinner("🧠 Performing Hybrid Search & Multi-Dimensional Scoring..."):
                    final_reports = system.run_matching_pipeline()

            status_container.empty()
            st.balloons()

            st.subheader(f"🏆 Top Candidates ({len(final_reports)} Matches Found)")

            if not final_reports:
                st.error("No suitable candidates found based on the current criteria.")

            for i, report in enumerate(final_reports):
                score = report.get('final_score', 0)

                if score >= 85:
                    score_color = "green"
                    verdict = "🌟 Highly Recommended"
                    border_color = "#2ecc71"
                elif score >= 70:
                    score_color = "blue"
                    verdict = "✅ Recommended"
                    border_color = "#3498db"
                elif score >= 50:
                    score_color = "orange"
                    verdict = "⚠️ Consider"
                    border_color = "#f39c12"
                else:
                    score_color = "red"
                    verdict = "❌ Not Recommended"
                    border_color = "#e74c3c"

                with st.container():
                    st.markdown(f"""
                    <div class="report-card" style="border-left: 5px solid {border_color};">
                        <div style="display: flex; justify-content: space-between; align-items: center;">
                            <h3 style="margin:0;">#{i+1} {report.get('name', 'Unknown Candidate')}</h3>
                            <div style="text-align: right;">
                                <h2 style="margin:0; color: {border_color};">{score:.1f}%</h2>
                                <span style="background-color: {border_color}; color: white; padding: 2px 8px; border-radius: 10px; font-size: 0.8em;">{verdict}</span>
                            </div>
                        </div>
                        <p style="color: gray; font-size: 0.9em;">File: {report.get('filename', 'N/A').split('/')[-1]}</p>
                    </div>
                    """, unsafe_allow_html=True)

                    tab1, tab2, tab3 = st.tabs(["📝 AI Analysis", "📊 Score Breakdown", "🔍 Raw Data"])

                    with tab1:
                        col1, col2 = st.columns(2)
                        with col1:
                            st.markdown("#### ✅ Strengths")
                            st.info(report.get('strengths', 'No analysis available.'))
                        with col2:
                            st.markdown("#### ⚠️ Gaps")
                            st.warning(report.get('gaps', 'No analysis available.'))

                        st.markdown("#### 💡 Recommendation")
                        st.success(report.get('notes', 'No recommendation available.'))

                    with tab2:
                        st.markdown("#### Scoring Dimensions")
                        cols = st.columns(3)
                        metrics = [
                            ("Must-Have Skills", int(report.get('must_have_score', 0))),
                            ("Important Skills", int(report.get('important_score', 0))),
                            ("Experience Match", int(report.get('experience_score', 0))),
                            ("Recency", int(report.get('recency_score', 0))),
                            ("Domain Match", int(report.get('domain_score', 0))),
                            ("Nice-to-Haves", int(report.get('nice_to_have_score', 0)))
                        ]

                        for idx, (label, value) in enumerate(metrics):
                            with cols[idx % 3]:
                                st.metric(label, f"{value}/100")
                                st.progress(min(value, 100) / 100)

                    with tab3:
                        st.json(report)

                st.write("")

        except Exception as e:
            st.error(f"An unexpected error occurred: {str(e)}")
            st.exception(e)
else:
    st.info("👈 Please upload a Job Description and Candidate Resumes in the sidebar to start.")

    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown("""
        ### 🤖 Intelligent Parsing
        Uses LLMs to understand context, not just keywords. Extracts implicit skills and domains.
        """)
    with col2:
        st.markdown("""
        ### ⚖️ Hybrid Retrieval
        Combines semantic vector search with traditional keyword matching for best-in-class accuracy.
        """)
    with col3:
        st.markdown("""
        ### 📊 Explainable Scoring
        Provides detailed score breakdowns and AI-written explanations for every decision.
        """)

Writing app.py


# **Create Sample Files**

In [10]:

print("Creating sample files...")

# Sample Job Posting
job_posting_text = """
Job Posting: Senior Python Developer (FinTech)

We are seeking a Senior Python Developer with 5+ years of experience in building
scalable web applications for the financial technology (FinTech) sector.
The ideal candidate will be an expert in Python and the Django framework.

Responsibilities:
- Design and develop high-performance, reusable, and reliable Python code.
- Mentor junior developers and perform code reviews.
- Work with cloud services, preferably AWS.
- Build and maintain secure REST APIs.

Must-Have Qualifications:
- 5+ years of Python experience
- 3+ years with Django
- Strong knowledge of SQL and PostgreSQL
- B.S. in Computer Science

Important Qualifications:
- Experience with AWS (S3, EC2)
- Knowledge of Docker
- Experience with REST APIs

Nice-to-Have:
- Experience with React
- Familiarity with CI/CD pipelines
"""
with open("job_posting.txt", "w") as f: f.write(job_posting_text)

# --- Resume 1: Strong Match ---
resume_1_text = """
--- Jane Doe ---
Email: jane.doe@example.com

Summary:
Senior Software Engineer with 8 years of experience specializing in backend
development using Python and Django. Proven track record of leading teams
and delivering scalable e-commerce and FinTech solutions.

Experience:
- Senior Developer, FinBank (2019 - Present)
  - Led development of a new payment gateway using Python, Django, and PostgreSQL.
  - Deployed applications on AWS S3 and EC2.
  - Built secure REST APIs for mobile clients.
- Developer, TechCorp (2016 - 2019)
  - Built internal tools with Python.

Education:
- B.S. in Computer Science

Skills:
Python, Django, PostgreSQL, SQL, AWS, S3, EC2, Git, Docker, REST APIs
"""
with open("resume_strong_match.txt", "w") as f: f.write(resume_1_text)

# --- Resume 2: Weak Match (Wrong Domain) ---
resume_2_text = """
--- John Smith ---
Email: john.smith@example.com

Summary:
Frontend Developer with 3 years of experience. Passionate about creating
beautiful user interfaces with React. Eager to learn backend technologies.

Experience:
- Frontend Developer, WebStyles (2021 - Present)
  - Built and maintained the company's main website using React and Redux.
- Intern, Dev Co (2020)
  - Worked with JavaScript and HTML/CSS.

Education:
- B.A. in Graphic Design

Skills:
React, JavaScript, HTML, CSS, Redux, Git
"""
with open("resume_weak_match.txt", "w") as f: f.write(resume_2_text)

# --- Resume 3: Borderline Match (Missing Must-Haves) ---
resume_3_text = """
--- Priya Sharma ---
Email: priya.sharma@example.com

Summary:
Data Scientist with 6 years of experience in machine learning and data analysis.
Extensive experience with Python and its data stack (Pandas, NumPy).

Experience:
- Data Scientist, DataAnalytics Inc. (2018 - Present)
  - Built ML models using Python, Scikit-learn, and TensorFlow.
  - Deployed models on AWS Sagemaker.
- Data Analyst, Insights Co. (2016 - 2018)
  - Wrote SQL queries and created dashboards.

Education:
- M.S. in Data Science
- B.S. in Computer Science

Skills:
Python, SQL, Pandas, NumPy, Scikit-learn, TensorFlow, AWS, Sagemaker
"""
with open("resume_borderline.txt", "w") as f: f.write(resume_3_text)

print("Sample files created: job_posting.txt, resume_strong_match.txt, resume_weak_match.txt, resume_borderline.txt")

Creating sample files...
Sample files created: job_posting.txt, resume_strong_match.txt, resume_weak_match.txt, resume_borderline.txt


# **11. Run the Full End-to-End Pipeline**

In [17]:

import importlib
import config
import utils
import llm_interface
import retrieval
import scoring
import matching_system

importlib.reload(config)
importlib.reload(utils)
importlib.reload(llm_interface)
importlib.reload(retrieval)
importlib.reload(scoring)
importlib.reload(matching_system)

from matching_system import CandidateMatchingSystem

print("--- 🚀 STARTING CANDIDATE MATCHING PIPELINE (v2 with ChromaDB & Retries) 🚀 ---")

try:
    system = CandidateMatchingSystem()

    system.process_job_posting("job_posting.txt")
    print("\n--- ✅ Job Posting Parsed Successfully ---")
    print(json.dumps(system.job.model_dump(), indent=2))

    resume_files = ["resume_strong_match.txt", "resume_weak_match.txt", "resume_borderline.txt"]
    system.process_resumes(resume_files)
    print(f"\n--- ✅ {len(system.candidates_db)} Resumes Parsed Successfully ---")

    final_reports = system.run_matching_pipeline()

    print("\n\n--- 🏆 FINAL RANKED CANDIDATE REPORT 🏆 ---")

    for i, report in enumerate(final_reports):
        print("\n" + "="*50)
        print(f"      RANK #{i+1} - {report['filename']}")
        print(f"      CANDIDATE: {report['name']}")
        print(f"      OVERALL FIT: {report['final_score']}/100")
        print(f"      CONFIDENCE: {report.get('confidence', 'N/A')}")
        print("="*50)
        print(f"\n**Strengths:**\n{report.get('strengths', 'N/A')}")
        print(f"\n**Gaps:**\n{report.get('gaps', 'N/A')}")
        print(f"\n**Notes:**\n{report.get('notes', 'N/A')}")
        print("\n--- Scoring Breakdown ---")
        print(f"- Must-Haves: {report['must_have_score']:.0f}/100")
        print(f"- Important: {report['important_score']:.0f}/100")
        print(f"- Experience: {report['experience_score']:.0f}/100")
        print(f"- Recency: {report['recency_score']:.0f}/100")
        print(f"- Domain: {report['domain_score']:.0f}/100")
        print("\n" + "-"*50)

except Exception as e:
    print(f"\n--- FATAL ERROR---")
    print(e)

print("\n--- 🏁 PIPELINE COMPLETE 🏁 ---")

File 'config.py' created.
File 'utils.py' created.
File 'llm_interface.py' (Final Version with Import Fix) created.
File 'retrieval.py' (Upgraded with ChromaDB Fix) created.
File 'scoring.py' (Final Version with Corrected Logic) created.
File 'matching_system.py' (Final Robust Version) created.
--- 🚀 STARTING CANDIDATE MATCHING PIPELINE (v2 with ChromaDB & Retries) 🚀 ---
HybridRetriever initialized with ChromaDB.
ScoringEngine initialized.
CandidateMatchingSystem initialized.
Extracting text from: job_posting.txt
LLM Call: Pydantic parsing with mistralai/mistral-7b-instruct:free...
Successfully parsed job: Senior Python Developer (FinTech)

--- ✅ Job Posting Parsed Successfully ---
{
  "job_title": "Senior Python Developer (FinTech)",
  "required_years_experience": 5,
  "skills": {
    "must_have": [
      "Python",
      "Django",
      "SQL",
      "PostgreSQL",
      "B.S. in Computer Science"
    ],
    "important": [
      "AWS (S3, EC2)",
      "Docker",
      "REST APIs"
    ],


In [1]:
!pip install pyngrok -q


# **Run the Streamlit UI with ngrok**

In [10]:

from pyngrok import ngrok

ngrok.set_auth_token("Your Ngrok_Authtoken here...")

public_url = ngrok.connect(8501).public_url
print(f"🚀 Your App is Live at: {public_url}")

!streamlit run app.py >/dev/null

🚀 Your App is Live at: https://sadye-openchain-primely.ngrok-free.dev
